In [4]:
import gensim

from nltk.data import find
import nltk

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary = True)

In [30]:
import numpy as np
from ipywidgets import interact

In [ ]:
model.save('gnews.embedding')

In [5]:
new_model = gensim.models.KeyedVectors.load('gnews.embedding')

In [6]:
import nltk

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/trevnels/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

In [9]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize 

pad_to_length = 32
def split_sanitize(sentence):
    sentence = str(sentence)
    tokens = word_tokenize(sentence)
    filtered_sentence = [w for w in tokens if not w.lower() in stopwords]
    filtered_sentence += [""] * (pad_to_length - len(filtered_sentence))
    return filtered_sentence[:pad_to_length]

def embed_tokens(tokens):
    embedding = []
    for token in tokens:
        if token in new_model:
            embedding.extend(new_model[token])
        elif token.lower() in new_model:
            embedding.extend(new_model[token.lower()])
    embedding.extend((pad_to_length - len(embedding)//300) * ([0.0,] * 300))
    return embedding

spam = pd.read_csv('data/spam.csv', encoding = "utf-8")
notspam = pd.read_csv('data/notspam.csv', encoding = "latin")

X = []
y = []

for tweet in spam.iloc[0:2500, 7]:
    # print(str(tweet))
    embedded = embed_tokens(split_sanitize(str(tweet)))
    
    if len(embedded) != 9600:
        print(embedded)
        break
    
    X.append(embedded)
    y.append(1.0)
    
for tweet in notspam.iloc[0:2500, 5]:
    # print(str(tweet))
    embedded = embed_tokens(split_sanitize(str(tweet)))
    
    X.append(embedded)
    y.append(0.0)
    
    

In [ ]:
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(20, 10), random_state=1)

clf.fit(X,y)


